In [1]:
import faiss
import os
import pandas as pd
import numpy 
import glob
import numpy as np
from skimage import measure

In [2]:
NPZ_DELF_FEAT_SRC = "/local/cs572/rchan31/codespace/BaselinePrototype/npz_file_saves/COMPACT_NPZ_DELF_FEATS/"
DELF_FEAT_SRC = "/local/cs572/rchan31/codespace/DELF_FEATURE_EXTRACTION/DELF_FEATURES/"

In [3]:
npz_delf_feat_files = sorted(glob.glob(os.path.join(NPZ_DELF_FEAT_SRC, "*.npz"))) # 952134
delf_feat_files = sorted(glob.glob(os.path.join(DELF_FEAT_SRC, "*.delf"))) # 952134

In [6]:
print("len of npz_delf_feat_files : ", len(npz_delf_feat_files))
print("len. of delf_feat_files : ", len(delf_feat_files))
print("missing file at index %s with filename : %s" % (413692, delf_feat_files[413692].split("/")[-1]))
# NPZ is missing one file

len of npz_delf_feat_files :  952133
len. of delf_feat_files :  952134
missing file at index 413692 with filename : 6f5819e00cdd1acb.delf


# generate retrieval_solution available samples

In [59]:
PROCESSED_RETRIEVAL_SOLUTION_CSV = "/local/cs572/rchan31/fINALwEEK/csv_lookups/processed_retrieval_solutions.csv"

In [12]:
# find out available test images and their test image-features
quer_vecs = []
quer_ids = {}
for i, fn in enumerate(glob.glob('./COMPACT_NPZ_DELF_test_FEATS/*.npz')):
    ID = os.path.basename(fn)[:-4]
    vec = np.load(fn)["descriptors"]
    quer_vecs.append(np.asarray(vec[0, :], dtype=np.float32))
    quer_ids[i] = ID
quer_vecs = np.array(quer_vecs)

quer_vecs.shape

(667, 40)

In [20]:
all_ret_solution_df = pd.read_csv('/local/cs572/cziems/google-landmarks/retrieval_solution.csv')

solution_df = all_ret_solution_df[[ID in quer_ids.values() for ID in all_ret_solution_df['id']]]

solution_df.shape # (667, 3)

(667, 3)

In [50]:
retrieved_img_answers = set()
max_len = 0
solution_pairs = []
for sol in solution_df['images']:
    retrieved = sol.split(" ")
    
    max_len = max(0, len(set(retrieved)))
    retrieved_img_answers.update(set(retrieved))
    
    sol_arr = np.array(retrieved[:100] + ['None']*(100-len(retrieved[:100])))
    solution_pairs.append(sol_arr)
    

solution_pairs = np.array(solution_pairs)
solution_pairs.shape # (667, 100)

(667, 100)

In [51]:
print(max_len) # 307       ---> maximum length of solutions at each query-image
print(len(answers)) # 507  ---> total number of retrieval db-images in solution

307
507


In [55]:
df = pd.DataFrame(solution_pairs)
df.insert(0, "id", solution_df["id"].tolist())
df.head()

,id,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,f60a52c609ab0208,03548a203d4b07f5,03a75dd20de880e3,04269e168c004405,04ceea28094f5758,054e6f31bdf12407,0555aac7133acbf1,057dc023b9414fe1,05a00237c243356c,0733a0a44489474d,...,3586fd1e2703fe16,35ec6ef9946e40d6,3766944fab5c3d1a,37ebb0e06f5be51f,38eba56c2498fdcf,39192a6d0d690e45,3a125000b1ea0de5,3ab34fe9d1dfcd15,3bb176e4dfd77846,3c428ddc62d721a4
1,4fa678d60ab4bc23,007eda76138e36e1,03edf3db543d2973,1c37cf230d45cf03,1e987a74ef9941bd,2eb8a14efa2f8724,2f3e7fe2aba15989,3869174c22bba91f,3873f493c6805bee,39fb16e9e506d285,...,None,None,None,None,None,None,None,None,None,None
2,5be9d8002f8dc60e,00c4c92c7a8fd255,03a7c829d0b89c83,04c5c412e3ceec4a,0a28ea1e9de51440,0b6b90750c7b328d,0c37aad42a99d536,0df10bece1f6c632,130485c0f709ef35,1679a35ef83f9333,...,None,None,None,None,None,None,None,None,None,None
3,0810084dfbce8ea8,08988c4cdb26f424,094a5e4ff1010550,421b161412e67a56,535c841fe0a36d1d,5ce9b333457b76e5,65313bf0521089c7,ace287038a3f52dd,b64fd516d9eebd5e,None,...,None,None,None,None,None,None,None,None,None,None
4,bb8ca55b60e8cff4,1e1533bb6e455633,1e58c6380e4db235,373e487e2562f02e,522d403681efdfee,5d30abd250225062,7b94bb1f87773b98,7ee82dfb4eaa3668,ad2ecd8c4a2d02d1,afe66a1f9cf8f8d9,...,None,None,None,None,None,None,None,None,None,None


In [60]:
df.to_csv(PROCESSED_RETRIEVAL_SOLUTION_CSV)